Integrating spherical Voronoi code into scipy.spatial will require removing the dependency on pandas, among other things. The purpose of this notebook is to allow interactive testing as I adjust the spherical Voronoi code accordingly and to compare performance / behaviour back to the original code in my own repo.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy
import scipy
import time
%load_ext autoreload
%autoreload 2
from scipy.spatial.spherical_voronoi import SphericalVoronoi #eventually I want to be able to import this directly from scipy.spatial
import sys; sys.path.append('/Users/treddy/github_projects/py_sphere_Voronoi') #so I can import the code from my own repo for comparison with the scipy-integrated code
import voronoi_utility #my own repo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#generate a random array of 5000 points on the surface of a unit sphere:
prng = numpy.random.RandomState(117) #pin down the random seed
random_coords_on_sphere = voronoi_utility.generate_random_array_spherical_generators(5000, 1.0, prng)

In [26]:
#run the original spherical Voronoi code using the above generators (this should work just fine):
original_voronoi_instance = voronoi_utility.Voronoi_Sphere_Surface(random_coords_on_sphere,1.0)
dictionary_voronoi_polygon_vertices = original_voronoi_instance.voronoi_region_vertices_spherical_surface()

In [31]:
#likewise with the scipy-integrated spherical Voronoi code (this now works after replacing pandas operations with zip() and setdefault dict operations):
scipy_voronoi_instance = SphericalVoronoi(random_coords_on_sphere, 1.0)
scipy_dictionary_voronoi_polygon_vertices = scipy_voronoi_instance.voronoi_region_vertices_spherical_surface()

In [35]:
#given that all unit tests are already passing for the scipy-integrated pandas-free spherical Voronoi code, rather than plotting the spherical Voronoi diagrams to show they are the same, simply show that the dictionaries of polygon vertices match:
common_voronoi_regions = 0
for generator_index, array_polygon_vertices in dictionary_voronoi_polygon_vertices.iteritems():
    scipy_array_polygon_vertices = scipy_dictionary_voronoi_polygon_vertices[generator_index]
    if numpy.all([array_polygon_vertices, scipy_array_polygon_vertices]):
        common_voronoi_regions += 1
    
print 'common_voronoi_regions:', common_voronoi_regions
print 'number of generators:', len(dictionary_voronoi_polygon_vertices.keys())

common_voronoi_regions: 4999
number of generators: 4999


The dictionaries of Voronoi regions match and all unit tests are passing for the scipy-integrated pandas-free code, so we have pretty high confidence in the scipy-migrated code. Now, benchmark the relative performance of the two sets of spherical Voronoi code, to see if I've lost any efficiency by dropping pandas in the scipy version.